In [ ]:
import numpy as np
np.random.seed(1337)  # for reproducibility
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,recall_score,confusion_matrix,f1_score,precision_score
from sklearn.metrics import cohen_kappa_score,hamming_loss,precision_recall_curve
from sklearn.metrics import roc_curve,auc
from sklearn.metrics import classification_report
from matplotlib import pyplot as plt
from DBN.models import SupervisedDBNClassification
import tensorflow._api.v2.compat.v1 as tf
#import tensorflow.compat.v1 as tf ###
#tf.disable_v2_behavior()
import  pandas as pd
#import eli5
#from eli5.sklearn import PermutationImportance
#import shap
from sklearn.preprocessing import normalize
from sklearn.preprocessing import MinMaxScaler

In [ ]:
# Loading dataset
# digits = load_digits()
# X, Y = digits.data, digits.target
#data = pd.read_csv(r'Ctrain.csv',sep=',')
data = pd.read_csv('test_files.csv',sep=',')
data_= pd.read_csv('predict_files.csv',sep=',')
data_.dropna(inplace=True)
label_index = 7
dim  = 7
# data[label_index] = data[label_index].map({1:1, 2:-1})
data = data.sample(frac=1.0)
data = data.reset_index()
x = data.drop(['landslide'],axis=1)
print(x.shape)
x = x.drop(['index'],axis=1)
x = np.array(x)
print(x.shape)
std=MinMaxScaler((0,1))
x=std.fit_transform(x)
y = data['landslide']
y = pd.to_numeric( y, errors='coerce').fillna('0').astype('int32')
# Data scaling
# X = (X / 16).astype(np.float32)

In [ ]:
data_ = data_.sample(frac=1.0)
data_ = data_.reset_index()
x_ = data_.drop(['landslide','pre1'],axis=1)
x_ = x_.drop(['index'],axis=1)
x_ = np.array(x_)
res=x_
print(x_.shape)
std=MinMaxScaler((0,1))
x_=std.fit_transform(x_)
y_ = data_['landslide']
y_ = pd.to_numeric( y_, errors='coerce').fillna('0').astype('int32')

In [ ]:
# Splitting data
X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size=0.3, random_state=1)#random_state=0
X_train[:,2:]
# Training
classifier = SupervisedDBNClassification(hidden_layers_structure=[100, 100],
                                         learning_rate_rbm=0.07, 
                                         learning_rate=0.15,
                                         n_epochs_rbm=16,
                                         n_iter_backprop=500,
                                         batch_size=20,
                                         activation_function='relu',
                                         dropout_p=0)
classifier.fit(X_train[:,2:], Y_train)

In [ ]:
# Test
Y_pred = classifier.predict(X_test[:,2:])

Y_pred_p = classifier.predict_proba(X_test[:,2:])[:,1]
#Y_pred_p = classifier.predict_proba_dict(X_test[:,2:])[:]

Y_pred = np.array(Y_pred)
Y_test_1 = np.array(Y_test)
##Y_pred_p = np.array(Y_pred_p)
Y_pred_1 = np.reshape(Y_pred,(len(Y_pred),1))
Y_test_1 = np.reshape(Y_test_1,(len(Y_test_1),1))
##Y_pred_p = np.reshape(Y_pred_p,(len(Y_pred_p),1))
#Y_pred_1

print('Done.\nAccuracy: %f' % accuracy_score(Y_test, Y_pred))
print('Done.\nprecision_score: %f' % precision_score(Y_test, Y_pred))
print('Done.\nrecall_score: %f' % recall_score(Y_test, Y_pred))
print('Done.\nf1_score: %f' % f1_score(Y_test, Y_pred))
print('Done.\nconfusion_matrix: ' )
print(confusion_matrix(Y_test, Y_pred))
print('Done.\ncohen_kappa_score: %f' % cohen_kappa_score(Y_test, Y_pred))
print('Done.\nhamming_loss: %f' % hamming_loss(Y_test, Y_pred))
fpr,tpr,threshold= roc_curve(Y_test,Y_pred_p)
#print('Done.\nauc: %f' % auc(fpr, tpr))
##target_names = ['class 0','class 1','class 2', 'class 3']
##print(classification_report(Y_test, Y_pred, target_names=target_names))

In [ ]:
# Test
y_pred = classifier.predict(x_[:,2:])
y_pred_p = classifier.predict_proba(x_[:,2:])[:,1]
y_pred = np.array(y_pred)
y_test_1 = np.array(y_)
##y_pred_p = np.array(y_pred_p) 
y_pred_1 = np.reshape(y_pred,(len(y_pred),1))
y_test_1 = np.reshape(y_test_1,(len(y_test_1),1))
##y_pred_p = np.reshape(y_pred_p,(len(y_pred_p),1))

In [ ]:
all_save_data = np.concatenate((res,y_test_1),axis=1)
mode=0
if mode==0:
    all_save_data = np.concatenate((all_save_data,y_pred_1),axis=1)
else:
    all_save_data = np.concatenate((all_save_data,y_pred_p),axis=1)
all_save_data_pd = pd.DataFrame(all_save_data,columns=['x','y','road','PGA',
 'river','fault','lithology','NDVI','Ht','landslide','predict'])
 # np.savetxt('data.csv',all_save_data,delimiter=',')
all_save_data_pd.to_csv('result.csv',index=True)
# y_test_1.shape

In [ ]:
precision, recall, thresholds = precision_recall_curve(np.array(Y_test),Y_pred_p)
plt.figure(figsize=(5,4),dpi=100)
plt.plot(fpr,tpr)
plt.plot([0,1],[0,1],linestyle='--')
plt.xlabel('False Postive Rate')
plt.ylabel('True Positive Rate')
plt.grid()
plt.show()
plt.figure(figsize=(5,4),dpi=100)
plt.plot(precision,recall)
plt.plot([0,1],[1,0],linestyle='--')
plt.xlabel('recall')
plt.ylabel('precision')
plt.grid()
plt.show()

In [ ]:
Y_pred = classifier.predict(X_train[:,2:])
Y_pred_p = classifier.predict_proba(X_train[:,2:])[:,1]
print('Done.\nconfusion_matrix: ' )
print(confusion_matrix(Y_train, Y_pred))
print('Done.\ncohen_kappa_score: %f' % cohen_kappa_score(Y_train, Y_pred))

fpr,tpr,threshold= roc_curve(Y_train,Y_pred_p)
precision, recall, thresholds = precision_recall_curve(np.array(Y_train),Y_pred_p)
plt.figure(figsize=(5,4),dpi=100)
plt.plot(fpr,tpr)
plt.plot([0,1],[0,1],linestyle='--')
plt.xlabel('False Postive Rate')
plt.ylabel('True Positive Rate')
plt.grid()
plt.show()
plt.figure(figsize=(5,4),dpi=100)
plt.plot(precision,recall)
plt.plot([0,1],[1,0],linestyle='--')#
plt.xlabel('recall')
plt.ylabel('precision')
plt.grid()
plt.show()